# ASG Warm Pool
- Autoscale에서 확장에 필요한 인스턴스를 미리 준비하는 서비스
    - 인스턴스의 준비 기간이 매우 긴 상황에서 Autoscaling의 확장 시 인스턴스를 빠르게 준비 가능
        - 즉 인스턴스 준비 기간이 길지 않을 경우 체감 효과는 미비
    - 대표 사용 사례 : 볼륨 용량이 매우 커서 AMI -> 인스턴스 준비 기간이 오래 걸리는 경우
- Warm Pool : Autoscale 상황을 위해 미리 준비해둔 인스턴스의 Pool
    - 3가지 상태
        - Running : 인스턴스 비용을 포함한 모든 비용 지불
        - Stopped / Hibernate(지원하는 경우) : EBS 비용과 IP 비용
- 재활용 정책
    - 기본적으로 Scale In 활동 시 인스턴스는 삭제됨
    - 단, Reuse Policy를 설정하여 다시 Warm Pool로 복귀시켜 재사용 가능
        - Warm Pool 생성 설정을 제외하고는 AWS CLI / SDK 로만 작동 가능
- EC2 뿐만 아니라 EKS / ECS 등에서도 활용 가능

## Hibernate(최대 절전 모드)
- 단순하게 인스턴스만 중지하는 것이 아니라 메모리의 상태 역시 데이터화해서 저장해둔 상태
    - 즉 어플리케이션을 처음부터 시작하는 것이 아니라 중단 시점부터 재개 가능

## Autoscale Warm Pool의 크기
- 기본적으로 웜 풀의 크기는 오토스케일 그룹의 Max Capacity - Desired Capacity
    - EX. Min / Desired / Max = 1 / 3 / 8 이라면 웜풀 사이즈는 5
- MaxGroupPreparedCapacity를 별도로 설정하여 크기 고정 가능
    - 예 : Desired / Max 가 1000 / 1500 이라면 500개의 웜풀 사이즈는 과할 수도 있음

## AUtoscale Warm Pool Flow
- Lifecycle Hook을 활용하여 추가적인 설정 가능
    - EC2_INSTANCE_LAUNCHING : WarmPool 진인 시 / 실제로 Live로 올라갈 때 2번 실행
    - EC2_INSTANCE_TERMINATING : 종료되거나 WarmPool로 돌아갈 때 실행
- Lifecycle Hook이 없다면 Warm Pool 진입 후 Userdata 수행 중 Stop / Hibernate 될 수 있음

### 데모
1. IAM 역할 생성
    - EC2가 사용
    - 인라인 권한 생성, JSON 스크립트 입력
2. EC2 시작 템플릿 생성
    - 키페어x
    - 보안그룹 디폴트
    - 고급 세부 정보에 IAM 프로파일 적용
    - 유저데이터에 스크립트 입력
3. ASG 생성
    - 이름 : demo-asg-warmpool
    - 원하는 용량 0, 최소 용량 0, 최대 용량 2
    - 태그 Name 입력
4. Lifecycle Hook 생성
    - 이름 : on-instance-start
    - 하트비트 제한 시간 300초
5. ASG Desired Capacity 조정
    - 1로 올리고 웜풀 생성
        - 중지됨, 스케일 인 재사용
    - 2로 올려보자
        - 인스턴스가 빠르게 준비되는 것을 확인할 수 있음
    - 1로 다시 줄이면?
        - 다시 웜풀로 돌아가겠지